In [ ]:
import math
from datetime import datetime,timedelta

def gen_train(volume, weather, tid, direct, time_int, dmin=-1, dmax = -1, test = 0):
    data = [[]]
    wmin = np.array([weather[i] for i in sorted(weather.keys())]).min(axis=0)
    wmax = np.array([weather[i] for i in sorted(weather.keys())]).max(axis=0)
    wres = [[]]
    vkeys = list(volume.keys())
    vkeys.sort()
    pre_day =  vkeys[0].day
    if test:
        delta = timedelta(hours=2)
    else:
        delta = timedelta(hours=0)
    for i in vkeys:
        
        current_hour = i.hour
        current_day = i.day
        if current_day != pre_day:
            data.append([])
            wres.append([])
        if current_hour in time_int:
#             data[-1].append([volume[i][tid][direct]]+weather[i])
            data[-1].append([volume[i][tid][direct], i.weekday()/7.0])
            wres[-1].append(weather[i+delta])
        pre_day = current_day
        
    res = np.array(data, dtype = np.float64)
    if dmin == -1:
        dmin = res[:, :, 0].min()
    if dmax == -1:
        dmax = res[:, :, 0].max()
    res[:, :, 0] = (res[:, :, 0]-dmin)/(dmax-dmin)
    wres = (np.array(wres)-wmin)/(wmax-wmin)
    return np.concatenate([res, wres], axis=2), dmin, dmax

def avgVolume(in_file):
    # Step 1: Load volume data
    fr = open(in_file, 'r')
    fr.readline()  # skip the header
    vol_data = fr.readlines()
    fr.close()

    # Step 2: Create a dictionary to caculate and store volume per time window
    volumes = {}  # key: time window value: dictionary
    for i in range(len(vol_data)):
        each_pass = vol_data[i].replace('"', '').split(',')
        tollgate_id = each_pass[1]
        direction = each_pass[2]

        pass_time = each_pass[0]
        pass_time = datetime.strptime(pass_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = int(math.floor(pass_time.minute / 20) * 20)
#         print(pass_time)
        start_time_window = datetime(int(pass_time.year), int(pass_time.month), int(pass_time.day),
                                     int(pass_time.hour), int(time_window_minute), 0)
#         print(start_time_window)
        if start_time_window not in volumes:
            volumes[start_time_window] = {}
        if tollgate_id not in volumes[start_time_window]:
            volumes[start_time_window][tollgate_id] = {}
        if direction not in volumes[start_time_window][tollgate_id]:
            volumes[start_time_window][tollgate_id][direction] = 1
        else:
            volumes[start_time_window][tollgate_id][direction] += 1
    # Step 3: format output for tollgate and direction per time window
    time_windows = list(volumes.keys())
    time_windows.sort()
    current = time_windows[0]
    tid = ['1', '2', '3']
    direct = {'1':['0', '1'], '2':['0'], '3':['0', '1']}
    while(1):
        if current not in volumes.keys():
            volumes[current] = {}
        for i in tid:
            if i not in volumes[current].keys():
                volumes[current][i] = {}
            for j in direct[i]:
                if j not in volumes[current][i].keys():
                    volumes[current][i][j] = 0
        if current == time_windows[-1]:
            break
        current = current + timedelta(minutes=20)
    return volumes
            
def clean_weather(f):
    f = open(f, 'r').readlines()
    f = [i.replace('"', '').split(',') for i in f]
    weather = {}
    cnt = 1
    start_time = datetime.strptime(f[cnt][0], "%Y-%m-%d")
    start_time = datetime(int(start_time.year), int(start_time.month), int(start_time.day),
                                     int(f[cnt][1]), 0, 0)
    current = start_time
    breakf = False
    while(1):
        if breakf and current == next_start:
            break
        if cnt == len(f)-1:
            next_start = start_time + timedelta(hours=3)
            breakf = True
        else:
            next_start = datetime.strptime(f[cnt+1][0], "%Y-%m-%d")
            next_start = datetime(int(next_start.year), int(next_start.month), int(next_start.day),
                                     int(f[cnt+1][1]), 0, 0)
        if current == next_start:
            start_time = current
            cnt += 1
        weather[current] = [float(i) for i in f[cnt][2:]]
        current = current + timedelta(minutes=20) 
    weather[current] = weather[current-timedelta(minutes=20)]
    return weather

weather = clean_weather("dataSets/training/weather (table 7)_training.csv")
volume = avgVolume("dataSets/training/volume(table 6)_training.csv")
tvolume = avgVolume("dataSets/testing_phase1/volume(table 6)_test1.csv")
tweather = clean_weather("dataSets/testing_phase1/weather (table 7)_test1.csv")
rvolume = avgVolume("dataSet_phase2/volume(table 6)_training2.csv")
rweather = clean_weather("dataSet_phase2/weather (table 7)_2.csv")
ttvolume = avgVolume("dataSet_phase2/volume(table 6)_test2.csv")

weather.update(rweather)
weather.update(tweather)

from __future__ import print_function
import math
import numpy as np
import torch
import torch.nn as nn 
from torch.autograd import Variable
import torch.optim as optim
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self, num = 128):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(2, num)
        self.lstm2 = nn.LSTMCell(num, num)
        self.lstm3 = nn.LSTMCell(num, 1)
        self.num = num

    def forward(self, input, future = 0, weather=None):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), self.num).double(), requires_grad=False).cuda()
        c_t = Variable(torch.zeros(input.size(0), self.num).double(), requires_grad=False).cuda()
        h_t2 = Variable(torch.zeros(input.size(0), self.num).double(), requires_grad=False).cuda()
        c_t2 = Variable(torch.zeros(input.size(0), self.num).double(), requires_grad=False).cuda()
        h_t3 = Variable(torch.zeros(input.size(0), 1).double(), requires_grad=False).cuda()
        c_t3 = Variable(torch.zeros(input.size(0), 1).double(), requires_grad=False).cuda()
        
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            input_tt = input_t.squeeze()
            h_t, c_t = self.lstm1(input_tt, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(c_t2, (h_t3, c_t3))
            outputs += [c_t3]
        for i in range(future):# if we should predict the future
            input_tt = torch.cat([c_t3.view(input.size(0), 1), weather[:, i].contiguous().view(input.size(0), 1)], dim=1)
            h_t, c_t = self.lstm1(input_tt, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(c_t2, (h_t3, c_t3))
            outputs += [c_t3]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

def lstm(num, lr):
    print('training on', num, lr)
    d={'1':['0', '1'], '2':['0'], '3':['0', '1']}
    mape = []
    f = open('volume2L_%d.csv'%num, 'w')
    f.write("tollgate_id,time_window,direction,volume\n")
    z = open('volume2Lv_%d.csv'%num, 'w')
    z.write("tollgate_id,time_window,direction,volume\n")
    interval = [[[6, 7, 8, 9], [6, 7, 15, 16, 17, 18]],
                [[6, 7], [6, 7, 15, 16]],
                [[8, 9], [17, 18]],
                [[6, 7], [6, 7, 15, 16]],
                [[6, 7], [15, 16]]]
    for time_int in range(2):
        for tid in ['1', '2', '3']:
            for direct in d[tid]:
                while(1):
                    print('tid:', tid, 'direction:', direct, interval[0][time_int][-4], interval[0][time_int][-4]+4)
                    seq = Sequence(num)
                    seq.double()
                    seq.cuda()
                    tmape = 10

                    train_data, dmin, dmax = gen_train(volume, weather, tid, direct, interval[0][time_int])
                    val_data, _ , _  = gen_train(tvolume, weather, tid, direct, interval[1][time_int], dmin, dmax)
                    val_target, tmin, tmax = gen_train(rvolume, weather, tid, direct, interval[2][time_int])
                    test_data, _ , _ = gen_train(ttvolume, weather, tid, direct, interval[3][time_int], dmin, dmax)
                    input = Variable(torch.from_numpy(train_data[:, :-1, 0:2]), requires_grad=False).cuda()
                    target = Variable(torch.from_numpy(train_data[:, 1:, 0]), requires_grad=False).cuda()
                    val = Variable(torch.from_numpy(val_data[:, :, 0:2]), requires_grad=False).cuda()
                    val_target = val_target*(tmax-tmin)+tmin
                    val_target = val_target[:, :, 0]
                    test = Variable(torch.from_numpy(test_data[:, :, 0:2]), requires_grad=False).cuda()
                    
                    val_weather, _, _ = gen_train(tvolume, weather, tid, direct, interval[4][time_int], dmin, dmax, 1)
                    test_weather, _ , _ = gen_train(ttvolume, weather, tid, direct, interval[4][time_int], dmin, dmax, 1)
                    valw = Variable(torch.from_numpy(val_weather[:, :, 1]), requires_grad=False).cuda()
                    testw = Variable(torch.from_numpy(test_weather[:, :, 1]), requires_grad=False).cuda()
                    
                    criterion = nn.SmoothL1Loss().cuda()
                    optimizer = optim.LBFGS(seq.parameters(), lr = lr)
                    for i in range(50):
                        print('step:', i,)
                        def closure():
                            optimizer.zero_grad()
                            out = seq(input)
                            loss = criterion(out, target)
#                             print('loss:', loss.data.numpy()[0])
                            loss.backward()
                            return loss
                        optimizer.step(closure)
                        future = 5
                        pred = seq(val, future = future, weather = valw)
                        y = pred.data.cpu()
                        y = y.numpy()
                        yy = y*(dmax-dmin)+dmin
                        fmape = np.mean(np.abs(yy[:, -6:]-val_target)/val_target)
                        if tmape>fmape:
                            tmape = fmape
                            best_yy = yy[:, -6:]
                            print(tmape)
                            res = seq(test, future = future, weather = testw)
                            res = res.data.cpu()
                            res = res.numpy()[:, -6:]
                            res = res*(dmax-dmin)+dmin
                        print("current mape:", fmape)

                    if np.max(np.abs(y))>0.1 and np.max(np.abs(y))<5 and tmape<0.20 and fmape < 1:
                        mape.append(tmape)
                        print('Choose')
                        print(tmape)
                        print(res-val_target)
                        stime = datetime(2016, 10, 25, interval[0][time_int][-4]+2, 0, 0)
                        for minute in range(6):
                            mtime = stime+timedelta(minutes=minute*20)
                            for day in range(7):
                                daytime = mtime+timedelta(days=day)
                                f.write("%s,\"[%s,%s)\",%s,%.2f\n"%(
                                    tid, 
                                    daytime,
                                    daytime+timedelta(minutes=20),
                                    direct,
                                    res[day, minute]
                                ))
                                z.write("%s,\"[%s,%s)\",%s,%.2f\n"%(
                                    tid, 
                                    daytime-timedelta(days=7),
                                    daytime+timedelta(minutes=20)-timedelta(days=7),
                                    direct,
                                    best_yy[day, minute],
                                ))
                        break
    for i in mape:
        f.write(str(i)+',')
    f.close()
    return mape

mape64 = lstm(64, 0.2)
mape128 = lstm(128, 0.2)
mape256 = lstm(256, 0.2)